In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
!pip install pyarrow

In [3]:
df_yellow_202301 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_yellow_202302 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
df_yellow = pd.concat([df_yellow_202301.assign(anomes=202301),df_yellow_202302.assign(anomes=202302)])
df_yellow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5980721 entries, 0 to 2913954
Data columns (total 21 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df_yellow['duration']=(df_yellow.tpep_dropoff_datetime - df_yellow.tpep_pickup_datetime).dt.total_seconds()/60
df_yellow['PU_DO'] = df_yellow.PULocationID.astype('str') + '_' + df_yellow.DOLocationID.astype('str')

In [6]:
## Q1
print(len(df_yellow_202301.columns))

19


In [7]:
##Q2
df_yellow.query("anomes == 202301").duration.std()

42.594351241920904

In [8]:
##Q3
df_yellow_ajus = df_yellow.query("duration >= 1 and duration <= 60").reset_index(drop=True)
print(df_yellow_ajus.shape[0]/df_yellow.shape[0])
del df_yellow

0.9806717283752243


In [9]:
##Q4
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
target = ['duration']

df_yellow_ajus[categorical] = df_yellow_ajus[categorical].astype('str')
train_dicts = df_yellow_ajus.query("anomes == 202301")[categorical+numerical].to_dict(orient='records')
y_train = df_yellow_ajus.query("anomes == 202301")[target].values
#del df_yellow_ajus

dv = DictVectorizer()
x_train = dv.fit_transform(train_dicts)
del train_dicts

x_train

<3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format>

In [10]:
##Q5
lr = LinearRegression()
lr.fit(x_train,y_train)
y_pred = lr.predict(x_train)
rmse = np.sqrt(mean_squared_error(y_pred,y_train))
del y_pred
rmse

7.649127185256247

In [11]:
##Q6

test_dicts = df_yellow_ajus.query("anomes == 202302")[categorical+numerical].to_dict(orient='records')
y_test = df_yellow_ajus.query("anomes == 202302")[target].values
x_test = dv.transform(test_dicts)
y_pred = lr.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_pred,y_test))
rmse

7.811475010118899